In [ ]:
import numpy as np
import csv
import random
from datetime import datetime
import sys
sys.path.insert(0,'C:/Users/patip/Documents/Python/Algorithm')
from Benchmark.functionbenchmark import HappyCat as myfunc

In [ ]:
dimension = 30
populationsize = 100
lowerbound = -100 
upperbound = 100
round = 20000
minimized_value  = 0
maximized_value = 0
minuted = 0
minimized_value_round = np.zeros(round)
minimized_vector = np.zeros(dimension)
minimized_value_keep = np.zeros((round,3))
value_analysis = np.zeros((round,3))
time_perround = np.zeros((10,1))
value = np.zeros(populationsize)
population = np.zeros((populationsize,dimension))
random_number = random.uniform(0,1)
min_round  = 0
generation = 0

for i in range(populationsize):
    dummy = np.random.uniform(low=lowerbound,high=upperbound+1,size=dimension)
    population[i:] = dummy

In [ ]:
def build_trial(r_vector1:np.array,r_vector2:np.array,r_vector3:np.array,target:np.array,fitness_max:float,fitness_mean:float,fitness_min:float,fitness_target:float,generation:float)->np.array:
    dummy_vector = np.random.uniform(low=0,high=1,size=dimension)
    """ Calculation F, alpha and CR rate """
    alpha = 1-random_number**((1-(generation/round))**2)
    if(fitness_mean != 0):
        scale_factor = (fitness_max-fitness_target)/fitness_mean
        cr_rate = (fitness_target-fitness_min)/fitness_mean
        mutant_vector = alpha*r_vector1+scale_factor*(r_vector2-r_vector3)
        trial = np.zeros(dimension)
        for i in range(dimension):
            if dummy_vector[i] > cr_rate:
                trial[i] = target[i]
            else:
                trial[i] = mutant_vector[i]
        return trial
    else:
        mutant_vector = alpha*r_vector1
        trial = np.zeros(dimension)
        for i in range(dimension):
            if dummy_vector[i] > 0:
                trial[i] = target[i]
            else:
                trial[i] = mutant_vector[i]
        return trial

def build_trial_best(best_vector:np.array,target:np.array,ration:float,fitness_max:float,fitness_mean:float,fitness_min:float,fitness_target:float,generation:float)->np.array:
    dummy_vector = np.random.uniform(low=0,high=1,size=dimension)
    if(fitness_mean != 0):
        scale_factor = (fitness_max-fitness_target)/fitness_mean
        cr_rate = (fitness_target-fitness_min)/fitness_mean
        mutant_vector = ration*best_vector+scale_factor*(best_vector-target)
        trial = np.zeros(dimension)
        for i in range(dimension):
            if dummy_vector[i] > cr_rate:
                trial[i] = target[i]
            else:
                trial[i] = mutant_vector[i]

    else:
        mutant_vector = ration*best_vector
        trial = np.zeros(dimension)
        for i in range(dimension):
            if dummy_vector[i] > 0:
                trial[i] = target[i]
            else:
                trial[i] = mutant_vector[i]
    
    return trial

def eveluate(fun,arg:np.array)->float:
    return fun(arg)

def selection(trial_vector:np.array,target_vector:np.array,fun)->np.array:
    trial_value = eveluate(fun,trial_vector)
    target_value = eveluate(fun,target_vector)
    """  print("Value")
    print(trial_value)
    print(target_value) """
    if trial_value <= target_value:
        """  print("Select new vector")
        print(trial_value) """
        return trial_vector
    else:
        """  print("Select original vector")
        print(target_value) """
        return target_vector




In [ ]:
from tqdm.notebook import tqdm_notebook
lowerbound = -100
upperbound = 100
for k in range(10):
    population = np.zeros((populationsize,dimension))
    for i in range(populationsize):
        dummy = np.random.uniform(low=lowerbound,high=upperbound+1,size=dimension)
        population[i:] = dummy

    now = datetime.now()
    for i in tqdm_notebook(range(round)):
        generation = i
        value = np.zeros(populationsize)
        population_dummy = np.zeros((populationsize,dimension))
        for j in range(populationsize):
            value[j] = eveluate(myfunc,population[j])
        
        fitness_max = max(value)
        fitness_min = min(value)
        fitness_mean = np.mean(value)
        if i == 0:
            index = np.where(value == fitness_min)[0][0]
            minimized_vector = population[index]
        
        for j in range(populationsize):
            fitness_target = value[j]
            ratio = fitness_min/fitness_target
            if ratio < 0.5:
                trial_vector = build_trial_best(best_vector=minimized_vector,target=population[j],ration=ratio,fitness_max=fitness_max,fitness_mean=fitness_mean,fitness_min=fitness_min,fitness_target=fitness_target,generation=i+1)
            else:
                vectorrand = np.random.permutation(populationsize)
                fitness_target = value[j]
                trial_vector = build_trial(population[vectorrand[0]],population[vectorrand[1]],population[vectorrand[2]],population[j],fitness_max=fitness_max,fitness_mean=fitness_mean,fitness_min=fitness_min,fitness_target=fitness_target,generation=i+1)
            
            population_dummy[j] = selection(trial_vector=trial_vector,target_vector=population[j],fun=myfunc)
            #value[j] = eveluate(myfunc,population[j])
        
        for j in range(populationsize):
            population[j] = population_dummy[j]
            value[j] = eveluate(myfunc,population[j])
        
        now2 = datetime.now()
        if i == 0:
            minimized_value = min(value)
            maximized_value = max(value)
            minimized_value_round[i] = minimized_value
            index = np.where(value == minimized_value_round[i])[0][0]
            minimized_vector = population[index]
            minuted = (now2-now).seconds
            min_round = i
            minimized_value_keep[i][0] = minimized_value
            minimized_value_keep[i][1] = min_round
            minimized_value_keep[i][2] = minuted
            
            #for analys solution
            value_analysis[i][0] = maximized_value;
            value_analysis[i][1] = minimized_value;
            value_analysis[i][2] = np.mean(value)
        else:
            if maximized_value > max(value):
                maximized_value = max(value)
            
            if minimized_value > min(value):
                minimized_value = min(value)
                minimized_value_round[i] = minimized_value
                index = np.where(value == minimized_value_round[i])[0][0]
                minimized_vector = population[index]
                minuted = (now2-now).seconds + ((now2-now).microseconds)*1E-6
                min_round = i
                minimized_value_keep[i][0] = minimized_value
                minimized_value_keep[i][1] = min_round
                minimized_value_keep[i][2] = minuted

                value_analysis[i][0] = maximized_value;
                value_analysis[i][1] = minimized_value;
                value_analysis[i][2] = np.mean(value)

            else:
                myindex = np.where(value == minimized_value)[0][0]
                #print(population[myindex])
                #print(minimized_vector)
                population[myindex:] = minimized_vector   #keep best to next generation
                minimized_value_round[i] = minimized_value
                minimized_value_keep[i][0] = minimized_value
                minimized_value_keep[i][1] = min_round
                minimized_value_keep[i][2] = minuted

                value_analysis[i][0] = maximized_value;
                value_analysis[i][1] = minimized_value;
                value_analysis[i][2] = np.mean(value)

    now3 = datetime.now()
    time_perround[k][0] = ((now3-now).seconds+((now3-now).microseconds)*1E-6)/round
    #print('round1 ok')
    file_path = 'C:/Users/patip/Documents/Python/Algorithm/raw_reault/HappyCat_result/Improve_DSIDE_HappyCat_'+str(dimension)+'_run_'+str(k+1)+'.csv'
    with open(file_path,'w',encoding='UTF8',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Solution','round','time'])
        writer.writerows(minimized_value_keep)


    file_path = 'C:/Users/patip/Documents/Python/Algorithm/analysis_result/HappyCat_result/Improve_DSIDE_HappyCat_analysis_'+str(dimension)+'_run_'+str(k+1)+'.csv'
    with open(file_path,'w',encoding='UTF8',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['max','min','mean'])
        writer.writerows(value_analysis)

file_path = 'C:/Users/patip/Documents/Python/Algorithm/time/Improve_DSIDE_HappyCat_timeperround_'+str(dimension)+'.csv'
with open(file_path,'w',encoding='UTF8',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['time_perround'])
        writer.writerows(time_perround)
    

    


In [ ]:
import pandas as pd
import csv

row_data = 20000
replace = 10

solution = np.zeros((replace,row_data))
solution_average = np.zeros((1,row_data))
minsolution = np.zeros(replace)
timecalculation = np.zeros(replace)
roundstable = np.zeros(replace)

for k in range(replace):
    path = 'C:/Users/patip/Documents/Python/Algorithm/raw_reault/HappyCat_result/Improve_DSIDE_HappyCat_'+str(dimension)+'_run_'+str(k+1)+'.csv'
    my_sol = pd.read_csv(path,nrows=row_data)
    solution[k:] = my_sol['Solution']
    solution[k:] = my_sol['Solution']
    minsolution[k] = min(my_sol['Solution'])
    roundstable[k] = max(my_sol['round'])
    timecalculation[k] = max(my_sol['time'])

for j in range(row_data):
    for i in range(replace):
        solution_average[0][j] = solution_average[0][j] + (solution[i][j]/replace)

solution_average = np.transpose(solution_average)
file_path = 'C:/Users/patip/Documents/Python/Algorithm/Improve_DSIDE_HappyCat_'+str(dimension)+'_.csv'
with open(file_path,'w',encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Average'])
    writer.writerows(solution_average)

print(minsolution)

print('Minimized Solution average: ',np.average(minsolution))
print('Minimized Solution standart diviation: ',np.std(minsolution))
print('Time find optimal solution: ',np.average(timecalculation)/60,' minuted')
print('Round find optimal solution: ',np.average(roundstable))

In [ ]:
result_analysis_max = np.zeros((replace,row_data))
result_analysis_min = np.zeros((replace,row_data))
result_analysis_mean = np.zeros((replace,row_data))

data_average = np.zeros((row_data,3))
for i in range(replace):
    path = 'C:/Users/patip/Documents/Python/Algorithm/analysis_result/HappyCat_result/Improve_DSIDE_HappyCat_analysis_'+str(dimension)+'_run_'+str(i+1)+'.csv'
    print(path)
    my_sol = pd.read_csv(path,nrows=row_data)
    result_analysis_max[i:] = my_sol['max']
    result_analysis_min[i:] = my_sol['min']
    result_analysis_mean[i:] = my_sol['mean']

for j in range(row_data):
    for i in range(replace):
        data_average[j][0] =data_average[j][0] + (result_analysis_max[i][j]/10)
        data_average[j][1] =data_average[j][1] + (result_analysis_min[i][j]/10)
        data_average[j][2] =data_average[j][2] + (result_analysis_mean[i][j]/10)




file_path = 'C:/Users/patip/Documents/Python/Algorithm/Improve_DSIDE_HappyCat_analysis_'+str(dimension)+'_.csv'
with open(file_path,'w',encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['max','min','mean'])
    writer.writerows(data_average)